The [scikit-learn](https://scikit-learn.org/stable/modules/naive_bayes.html) library support a number of Naive Bayes classifiers: 

- Gaussian: It assumes that continuous features follow a normal distribution.
- Multinomial: It is useful if your features are discrete.
- Bernoulli: The binomial model is useful if your features are binary.

### Titanic Survival Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

# Importing dataset
data = pd.read_csv("data/titanic_2.csv")

In [ ]:
data.head()

In [ ]:
# Convert categorical variable to numeric
data["Sex_cleaned"]=np.where(data["Sex"]=="male",0,1)
data["Embarked_cleaned"]=np.where(data["Embarked"]=="S",0,
                                  np.where(data["Embarked"]=="C",1,
                                           np.where(data["Embarked"]=="Q",2,3)
                                          )
                                 )
# Cleaning dataset of NaN
data=data[[
    "Survived",
    "Pclass",
    "Sex_cleaned",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "Embarked_cleaned"
]].dropna(axis=0, how='any')

In [ ]:
data.head()

In [ ]:
# Split dataset in training and test datasets
X_train, X_test = train_test_split(data, test_size=0.5, random_state=int(time.time()))

In [ ]:
# Instantiate the classifier
gnb = GaussianNB()
used_features =[
    "Pclass",
    "Sex_cleaned",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "Embarked_cleaned"
]

# Train classifier
gnb.fit(
    X_train[used_features].values,
    X_train["Survived"]
)
y_pred = gnb.predict(X_test[used_features])

# Print results
print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
      .format(
          X_test.shape[0],
          (X_test["Survived"] != y_pred).sum(),
          100*(1-(X_test["Survived"] != y_pred).sum()/X_test.shape[0])
))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
used_features =["Fare"]
y_pred = gnb.fit(X_train[used_features].values, X_train["Survived"]).predict(X_test[used_features])
print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
      .format(
          X_test.shape[0],
          (X_test["Survived"] != y_pred).sum(),
          100*(1-(X_test["Survived"] != y_pred).sum()/X_test.shape[0])
))
print("Std Fare not_survived {:05.2f}".format(np.sqrt(gnb.sigma_)[0][0]))
print("Std Fare survived: {:05.2f}".format(np.sqrt(gnb.sigma_)[1][0]))
print("Mean Fare not_survived {:05.2f}".format(gnb.theta_[0][0]))
print("Mean Fare survived: {:05.2f}".format(gnb.theta_[1][0]))

### Spam SMS

In this example, we will attempt to apply Naive Bayes on textual data. The target data set is [a collection of spam sms](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

The data table contains two columns separated by a tab. The two columns are **label** and **message**.

In [ ]:
import pandas as pd

df = pd.read_table('data/SMSSpamCollection',  
                   sep='\t', 
                   header=None,
                   names=['label', 'message'])
df.head()

Unlike numerical data, textual data requires additional processing. First, we need to convert the categorical labels into numerical values. 

In [ ]:
df['label'] = df.label.map({'ham': 0, 'spam': 1})
df.head()

Second, we need to convert all characters in the message to lower case:

In [ ]:
df['message'] = df.message.map(lambda x: x.lower())  
df.head()

Third, remove any punctuation:

In [ ]:
df['message'] = df.message.str.replace('[^\w\s]', '')  
df.head()

Fourth, tokenize the messages into into single words using nltk. First, we have to import and download the tokenizer from the console. 

To work with textual data, we will use the **nltk** library. NLTK is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, and wrappers for industrial-strength NLP (natural language processing) libraries.

In [ ]:
import nltk  
nltk.download('punkt')

In [ ]:
df['message'] = df['message'].apply(nltk.word_tokenize)
df.head()

Fifth, we will perform some word stemming. The idea of stemming is to normalize our text for all variations of words carry the same meaning, regardless of the tense. One of the most popular stemming algorithms is the Porter Stemmer:

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

df['message'] = df['message'].apply(lambda x: [stemmer.stem(y) for y in x])  
df.head()

Finally, we will transform the data into occurrences, which will be the features that we will feed into our model:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
df['message'] = df['message'].apply(lambda x: ' '.join(x))
df.head()

In [ ]:
count_vect = CountVectorizer()  
counts = count_vect.fit_transform(df['message'])  
print(len(count_vect.get_feature_names()))

We could leave it as the simple word-count per message, but it is better to use Term Frequency Inverse Document Frequency, more known as tf-idf:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)  

Training the Model
Now that we have performed feature extraction from our data, it is time to build our model. We will start by splitting our data into training and test sets:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], 
                                                    test_size=0.1, 
                                                    random_state=69)  

Then, all that we have to do is initialize the Naive Bayes Classifier and fit the data. For text classification problems, the Multinomial Naive Bayes Classifier is well-suited:

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)  

In [ ]:
Evaluating the Model
Once we have put together our classifier, we can evaluate its performance in the testing set:

In [ ]:
import numpy as np

predicted = model.predict(X_test)

print(np.mean(predicted == y_test))  

Congratulations! Our simple Naive Bayes Classifier has 98.2% accuracy with this specific test set! But it is not enough by just providing the accuracy, since our dataset is imbalanced when it comes to the labels (86.6% legitimate in contrast to 13.4% spam). It could happen that our classifier is over-fitting the legitimate class while ignoring the spam class. To solve this uncertainty, let's have a look at the confusion matrix:

### Question 2:

Complete the following code that apply Naive Bayes Classifier to a sentiment text data set. There are three datasets inside the sentiment_label_sentences for IMDB, Yelp, and Amazone review. You can select one to perform the classification process on. 

Hint, while this question is similar to the procedure of the Spam example, the structure of the data file is different. You need to review the data from the Spam example, and then identify the difference such that you can fill in the right contents. 

In [ ]:
import pandas as pd

df = pd.read_table('data/sentiment_label_sentences/________',  
                   sep='\t', 
                   header=None,
                   names=['_____', '_____'])
df.head()

In [ ]:
# convert to lower case
df[_____] = df._____.map(lambda x: x.lower())  

# remove punctuation
df[_____] = df._____.str.replace('[^\w\s]', '')  

# tokenize
import nltk  
nltk.download('punkt')
df[_____] = df[_____].apply(nltk._____)

# stem the words
from nltk.stem import _____
stemmer = _____()
df[_____] = df[_____].apply(lambda x: [stemmer.stem(y) for y in x])  
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings and then switch
# to term frequency format
df[_____] = df[_____].apply(lambda x: ' '.join(x))
df.head()

In [ ]:
count_vect = CountVectorizer()  
counts = count_vect.fit_transform(df[_____])  
print(len(count_vect.get_feature_names()))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)  
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, df[_____], 
                                                    test_size=0.1, 
                                                    random_state=69) 
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)  
import numpy as np

predicted = model.predict(X_test)

print(np.mean(predicted == y_test)) 